In [3]:
#librairy
import pandas as pd
import sqlite3

# 1.1 Movies Database

Take the movies dataset and turn it into a single `sqlite` database. It should have one table for each csv file in the movies dataset

In [4]:
df_movies = pd.read_csv('data/movies_metadata.csv')
df_credits = pd.read_csv('data/credits.csv')
df_keywords = pd.read_csv('data/keywords.csv')
df_links_small = pd.read_csv('data/links_small.csv')
df_ratings_small = pd.read_csv('data/ratings_small.csv')

In [7]:
df_movies.to_sql('movies', con = conn)
df_credits.to_sql('credits', con = conn)
df_keywords.to_sql('keywords', con = conn)
df_links_small.to_sql('links_small', con = conn)
df_ratings_small.to_sql('ratings_small', con = conn)

ValueError: Table 'movies' already exists.

In [6]:
conn = sqlite3.connect('data/movies.sqlite')

In [8]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", con=conn)

,name
0,movies
1,credits
2,keywords
3,links_small
4,ratings_small


# 1.2 Queries

**1.2.1** Use a single query to pull the original title of movies with a budget above $5m

**1.2.2** Use a query to pull the english-language films with the word `war` in their title

**1.2.3** Left join the average ratings from the `ratings` table onto the `movies_metadata` table, so you can have a relation between budget and rating. Hint: use a subquery.

In [8]:
#1.2.1 movie with a budget > 5m 

pd.read_sql("""
            SELECT original_title
            FROM movies
            WHERE budget > 5000000
            """, con=conn)

,original_title
0,Jumanji
1,Heat
2,Sabrina
3,GoldenEye
4,The American President
...,...
2182,The Emoji Movie
2183,Pattaya
2184,House of the Long Shadows
2185,Все и сразу


In [14]:
#1.2.2 pull english-language films with the  word war in their title 

pd.read_sql(
"""
SELECT
    original_title,
    original_language
FROM movies
WHERE
    (
    original_title LIKE '% war %'
    OR original_title LIKE '%war'
    OR original_title LIKE 'war%'
    OR original_title LIKE 'war'
    )
    AND original_language == 'en'
""", 
con=conn
)

,original_title,original_language
0,The War,en
1,The War Room,en
2,War Stories Our Mother Never Told Us,en
3,In Love and War,en
4,Warriors of Virtue,en
...,...,...
141,War for the Planet of the Apes,en
142,Underworld: Endless War,en
143,War Machine,en
144,Warning: This Drug May Kill You,en


In [15]:
#1.2.3 Left join the average ratings from the rating tables 

pd.read_sql(
"""
SELECT * 
FROM movies
JOIN (SELECT movieId, AVG(rating) as 'Average rating' 
FROM ratings_small 
GROUP BY movieId) as ratings ON ratings.movieId = movies.id
""", con = conn
)

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,movieId,Average rating
0,5,False,None,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",None,949,tt0113277,en,Heat,...,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,0,7.7,1886.0,949,3.593750
1,9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,...,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,0,6.6,1194.0,710,1.500000
2,14,False,None,98000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",None,1408,tt0112760,en,Cutthroat Island,...,119.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Course Has Been Set. There Is No Turning B...,Cutthroat Island,0,5.7,137.0,1408,3.616279
3,15,False,None,52000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",None,524,tt0112641,en,Casino,...,178.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,No one stays at the top forever.,Casino,0,7.8,1343.0,524,3.555556
4,16,False,None,16500000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",None,4584,tt0114388,en,Sense and Sensibility,...,136.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Lose your heart and come to your senses.,Sense and Sensibility,0,7.2,364.0,4584,5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2826,45318,False,None,0,"[{'id': 18, 'name': 'Drama'}]",None,80831,tt1576459,en,Sang Pemimpi,...,121.0,"[{'iso_639_1': 'id', 'name': 'Bahasa indonesia'}]",Released,None,Sang Pemimpi,0,6.5,2.0,80831,3.250000
2827,45353,False,"{'id': 123720, 'name': 'Frankenstein (Hammer S...",0,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",None,3104,tt0061683,en,Frankenstein Created Woman,...,92.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Now Frankenstein has created a beautiful woman...,Frankenstein Created Woman,0,5.9,33.0,3104,3.965517
2828,45403,False,None,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",None,64197,tt1047845,ru,Puteshestvie s domashnimi zhivotnymi,...,97.0,"[{'iso_639_1': 'ru', 'name': 'Pусский'}]",Released,None,Travelling with Pets,0,6.0,5.0,64197,4.166667
2829,45406,False,None,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",None,98604,tt2147597,ru,Zolushka,...,91.0,"[{'iso_639_1': 'ru', 'name': 'Pусский'}]",Released,None,Cinderella,0,4.6,6.0,98604,4.000000


# 2. Baseball Database

The [Baseball Database](http://www.seanlahman.com/baseball-archive/statistics/) has an sqlite version. Download it for these exercises.

**2.1** Which player has had the most homeruns?

**2.2** Is there a relation between how many homeruns a player has made in a year and his salary that year? Pull both colums together in a single query



In [20]:
conn = sqlite3.connect('data\lahmansbaseballdb (1).sqlite')
c = conn.cursor()


In [21]:
pd.read_sql("SELECT name FROM sqlite_MASTER WHERE type='table'", con=conn)

,name
0,allstarfull
1,appearances
2,awardsmanagers
3,awardsplayers
4,awardssharemanagers
5,awardsshareplayers
6,batting
7,battingpost
8,collegeplaying
9,divisions


In [41]:
#2.1 Which player has had the most homeruns?

df = (
    pd.read_sql_query(
        """
        SELECT playerID, SUM(HR) as HR FROM batting GROUP BY playerID
        """,
        conn
    )
)
#df.columns
df.loc[df.HR == df.HR.max()]
#Barry Bonds

,playerID,HR
1621,bondsba01,762


In [62]:
#2.2 Is there a relation between how many homeruns a player has made in a year and his salary that year? Pull both colums together in a single query


df = pd.read_sql_query(
    """
    SELECT  
        SUM(HR) as sum_HR,
        batting.yearID,
        batting.playerID,
        people.nameFirst,
        people.nameLast,
        salaries.salary
    FROM (batting
    INNER JOIN people
        ON
            batting.playerID = people.playerID
    INNER JOIN salaries
        ON
            batting.playerID = salaries.playerID
            AND batting.yearID = salaries.yearID
        )
    GROUP BY batting.playerID, batting.yearID
    ORDER BY sum_HR DESC
    """, conn)

df

,sum_HR,yearID,playerID,nameFirst,nameLast,salary
0,73,2001,bondsba01,Barry,Bonds,10300000.0
1,70,1998,mcgwima01,Mark,McGwire,8928354.0
2,66,1998,sosasa01,Sammy,Sosa,8325000.0
3,65,1999,mcgwima01,Mark,McGwire,9358667.0
4,64,2001,sosasa01,Sammy,Sosa,12500000.0
...,...,...,...,...,...,...
25495,0,2007,zumayjo01,Joel,Zumaya,410000.0
25496,0,2008,zumayjo01,Joel,Zumaya,420000.0
25497,0,2009,zumayjo01,Joel,Zumaya,735000.0
25498,0,2010,zumayjo01,Joel,Zumaya,915000.0


In [64]:
import statsmodels.api as sm 

X = df.sum_HR
y = df.salary

est = sm.OLS(y, sm.add_constant(X)).fit(cov_type='HC2')
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 salary   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     1085.
Date:                Wed, 17 Feb 2021   Prob (F-statistic):          4.35e-233
Time:                        08:03:54   Log-Likelihood:            -4.1915e+05
No. Observations:               25500   AIC:                         8.383e+05
Df Residuals:                   25498   BIC:                         8.383e+05
Df Model:                           1                                         
Covariance Type:                  HC2                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        1.51e+06   2.26e+04     66.948      0.000    1.47e+06    1.55e+06
sum_HR      1.137e+05   3450.307     32.941      0.000    1.07e+05     1.2e+05
==============================================================================
Omnibus:                    16297.767   Durbin-Watson:                   1.259
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           189434.488
Skew:                           2.980   Prob(JB):                         0.00
Kurtosis:                      14.949   Cond. No.                         12.0
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC2)
"""

Our r-squared is equal to 0.083, it doesn't seem to have a linear relation between the HR and salary paid. Based on that value we can say that most of the response data is around its mean. 